<a href="https://colab.research.google.com/github/UniLu-DMDB/UniLu-DMDB.github.io/blob/main/%5BData_Modeling_and_Database_Systems%5D_ParliamentDB_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

University of Lucerne

Data Modeling and Database Systems

# Exercises - ParliamentDB - SQL


## 1. Preparation

We start by setting up a connection to our Parliament database, Parliament.db.

In [1]:
# download SQLite DB
!wget https://unilu-dmdb.github.io/data/Parliament.db

--2024-02-15 15:30:36--  https://unilu-dmdb.github.io/data/Parliament.db
Resolving unilu-dmdb.github.io (unilu-dmdb.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to unilu-dmdb.github.io (unilu-dmdb.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 45056 (44K) [application/octet-stream]
Saving to: ‘Parliament.db’

Parliament.db       100%[===================>]  44.00K  --.-KB/s    in 0.01s   

2024-02-15 15:30:37 (3.33 MB/s) - ‘Parliament.db’ saved [45056/45056]



In [2]:
# load the SQL magic extension
# https://github.com/catherinedevlin/ipython-sql
# this extension allows us to connect to DBs and issue SQL command
%load_ext sql

# now we can use the magic extension to connect to our SQLite DB
# use %sql to write an inline SQL command
# use %%sql to write SQL commands in a cell
%sql sqlite:///Parliament.db

## 2. Exercises

Fill in the solutions and compare them to the results shown. If you want to see how to get to the results, click on "Show code" below the title "Solution".

### List the titles of all bills that have not yet been voted on (i.e., `status` is *introduced* or *debated*) ordered by the date of introduction (`DateOfInt`) in descending order.

In [ ]:
%%sql
<<YOUR SOLUTION>>

In [5]:
#@title Solution
%%sql
SELECT Title
FROM Bills
WHERE status IN ('introduced', 'debated')
ORDER BY DateOfInt DESC

 * sqlite:///Parliament.db
Done.


Title
The Eclair Appreciation Day Act
The Bubble Wrap Ban
The Onion Breath Regulation Act
The Maple Syrup Act
The Giraffe Height Limitation Act
The Syrup Sipping Incentive


### List all members of parliament (`FName`, `LName`) together with the name of their party (`Party`). Note that some MPs are not associated with any party and you should nevertheless include their names in the result list.

In [ ]:
%%sql
<<YOUR SOLUTION>>

In [6]:
#@title Solution
%%sql
SELECT FName, LName, Party
FROM MembersOFParliament
LEFT JOIN Parties USING (PID)

 * sqlite:///Parliament.db
Done.


FName,LName,Party
Isaac,Noodleton,Gravity Party
Gigglyeye,Galilei,Gravity Party
Winston,Waffle,Brunch Party
Samantha,Syrup,Brunch Party
Larry,Lettuce,None
Fiona,Fizzle,Muffin Party
Harry,Hiccup,Waffle Party
Oliver,Onion,Pancake Party
Gwendolyn,Giraffe,Pancake Party
Peter,Pickle,Brunch Party


### Return for each constituency the average age (making use of the _aggregation function_ `AVG(·)`) of its members of parliament.

In [ ]:
%%sql
<<YOUR SOLUTION>>

In [7]:
#@title Solution
%%sql
SELECT Constituency, AVG(Age) AS average_age
FROM MembersOFParliament
GROUP BY Constituency

 * sqlite:///Parliament.db
Done.


Constituency,average_age
Appleton,153.33333333333334
Bologna,179.0
Maple Ridge,38.666666666666664
Salad Town,49.666666666666664
Sugarland,36.0


### List for each party the number of bills introduced successfully by one of its members (`Status` is *passed*). You may ignore parties that have no successful bills or bills introduced by members that have no party association.

In [ ]:
%%sql
<<YOUR SOLUTION>>

In [8]:
#@title Solution
%%sql
SELECT Party, COUNT(DISTINCT BID) AS c
FROM Bills AS B
  INNER JOIN MembersOfParliament AS MP ON (B.IntBy = MP.MID)
  INNER JOIN Parties USING (PID)
WHERE
  status = 'passed'
GROUP BY
  Party

 * sqlite:///Parliament.db
Done.


Party,c
Brunch Party,1


### Return all members of parliament (all fields of the table `MembersOfParliament`) who have not cast any vote yet, i.e., there is no entry in `Votes` with their `MID`.

In [ ]:
%%sql
<<YOUR SOLUTION>>

In [9]:
#@title Solution
%%sql
SELECT *
FROM MembersOfParliament
WHERE
  MID NOT IN (SELECT MID FROM Votes)

 * sqlite:///Parliament.db
Done.


MID,FName,LName,Age,PID,Constituency
131,Maria,Ravioli,37,4,Bologna


### Return the (`DISTINCT`) title of the bills that i.) either are introduced ii.) or sponsored by a member of parliament without any party association (i.e., `PID` is `NULL`)

In [ ]:
%%sql
<<YOUR SOLUTION>>

In [10]:
#@title Solution
%%sql
SELECT Title
FROM Bills AS B
  INNER JOIN MembersOfParliament AS MP ON (B.IntBy = MP.MID)
WHERE
  PID IS NULL
UNION
SELECT Title
FROM Bills AS B
  INNER JOIN Sponsors AS S USING (BID)
  INNER JOIN MembersOfParliament AS MP USING (MID)
WHERE
  PID IS NULL

 * sqlite:///Parliament.db
Done.


Title
The Lettuce Liberation Act


### Find the top 3 bills (`BID`, `Title`) that have the highest number of sponsors. Order the results in descending order by the number of sponsors. To simplify, there is no need to handle ties in the number of sponsoring members.

In [ ]:
%%sql
<<YOUR SOLUTION>>

In [11]:
#@title Solution
%%sql
SELECT BID, Title, COUNT(DISTINCT MID) AS c
FROM Bills
  INNER JOIN Sponsors USING (BID)
GROUP BY
  BID, Title
ORDER BY
  c DESC
LIMIT
  3

 * sqlite:///Parliament.db
Done.


BID,Title,c
132,The Lettuce Liberation Act,4
76,The Pickle Preservation Act,2
42,The Anti-Gravity Act,1


### List all members of parliament (`MID`, `FName` and `LName`) that abstained (`Vote` is *abstention*) in at least 3 bills together with the number of abstentions.

In [ ]:
%%sql
<<YOUR SOLUTION>>

In [12]:
#@title Solution
%%sql
SELECT MID, FName, LName, COUNT(DISTINCT BID) AS c
FROM MembersOfParliament
  INNER JOIN Votes USING (MID)
WHERE
  vote = 'abstention'
GROUP BY
  MID, FName, LName
HAVING
  COUNT(DISTINCT BID) >= 3

 * sqlite:///Parliament.db
Done.


MID,FName,LName,c
3,Isaac,Noodleton,4
114,Roger,Rocket,3


### Return for each bill that was not passed by the parliament (`status` is *failed*) the number of yes and the number no votes (ignoring the abstentions). The result schema should look as follows (without the need to mimic the data):

\begin{array}{ccc}
BID&Title&Description&Status&yes&no\\
42&\text{The Anti-Gravity Act}&...&\text{failed}&34&52
\end{array}

In [ ]:
%%sql
<<YOUR SOLUTION>>

In [13]:
#@title Solution
%%sql
SELECT B.BID, Title, Description, Status, yes, no
FROM Bills AS B
  INNER JOIN (SELECT BID, COUNT(DISTINCT MID) AS yes FROM Votes WHERE Vote = 'yes' GROUP BY BID) USING (BID)
  INNER JOIN (SELECT BID, COUNT(DISTINCT MID) AS no FROM Votes WHERE Vote = 'no' GROUP BY BID) USING (BID)
WHERE
  status = 'failed'

 * sqlite:///Parliament.db
Done.


BID,Title,Description,Status,yes,no
103,The Rocket Propulsion Initiative,This bill aims to allocate funds for research and development of rocket propulsion technology for public transportation.,failed,2,8
154,The Waffle House Establishment Act,"This bill aims to provide government funding for the establishment of waffle houses in every neighborhood, ensuring access to delicious waffles for all.",failed,3,7
